**Federated learning**

Federated learning is a new paradigm in machine learning that allows the training of models on data from multiple sources without having to share the data. This is particularly important in scenarios where data privacy is a concern or where it is difficult to gather all the data in one place. This is a simple implementation of federated learning on MNIST data.

In [ ]:
 # Import relevant packages
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()


In [ ]:
# Normalize pixel values to between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0
# Reshape the data to add a channel dimension (required for convolutional layers)
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)


In [ ]:
plt.imshow(x_train[1]) # showing the data image


Define Model

In [ ]:
# CNN is used for digit classification
# Define the model architecture
def create_model():
    model = keras.Sequential([
        # Convolutional layer with 32 filters, a 3x3 filter size, and ReLU activation function
        keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        # Max pooling layer with a 2x2 pool size
        keras.layers.MaxPooling2D((2, 2)),
        # Add an other Convolutional layer with 64 filters, a 3x3 filter size, and ReLU activation function
        keras.layers.Conv2D(64, (3, 3), activation='relu'),
         # Add an other Max pooling layer with a 2x2 pool size
        keras.layers.MaxPooling2D((2, 2)),
        # Add an other Convolutional layer with 128 filters, a 3x3 filter size, and ReLU activation function
        keras.layers.Conv2D(128, (3, 3), activation='relu'),
         # Add an other Max pooling layer with a 2x2 pool size
        keras.layers.MaxPooling2D((2, 2)),
        # Flatten the output of the convolutional layers to prepare it for the dense layers
        keras.layers.Flatten(),
         # Add a dense layer with 128 units and ReLU activation function
        keras.layers.Dense(128, activation='relu'),
        # Add a dropout layer to prevent overfitting
        keras.layers.Dropout(0.5),
        # Output layer, this corresponds to a multiclass classification (softmax activation function) problem (output is from 0 to 9)
        keras.layers.Dense(10, activation='softmax')
    ])
    return model

**Simulating the Federated Learning Environment**

 To simulate the FL environment. We assume that there are three parties involved in the FL process, each with its own dataset. We randomly split the MNIST dataset into three parts and simulate the training process for each party.

In [ ]:
# Simulate the federated learning process
def federated_train(model, train_data, train_labels, test_data, test_labels, epochs, lr):
    for epoch in range(epochs):
        # Shuffle the data for each epoch
        indices = np.random.permutation(len(train_data))
        train_data = train_data[indices]
        train_labels = train_labels[indices]

        # Simulate the federated learning process for each party
        for party_id in range(3):
            # Get the local data and labels for the party
            party_data = train_data[party_id * (len(train_data) // 3):(party_id + 1) * (len(train_data) // 3)]
            party_labels = train_labels[party_id * (len(train_data) // 3):(party_id + 1) * (len(train_data) // 3)]

            # Train the local model
            model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
            model.fit(party_data, party_labels, epochs=1, verbose=0)

        # Evaluate the model on the test data
        test_loss, test_acc = model.evaluate(test_data, test_labels, verbose=0)

        print('Epoch', epoch + 1, '- Test loss:', test_loss, '- Test accuracy:', test_acc)

    return model


In [ ]:
# Define hyperparameters for the federated learning process
num_epochs = 10
learning_rate = 0.001

# Split the dataset into training and test sets
train_data, test_data, train_labels, test_labels = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

# Simulate the federated learning process
model = create_model()
model = federated_train(model, train_data, train_labels, test_data, test_labels, num_epochs, learning_rate)
